In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import googlemaps
import gmaps
import os
import requests
import shutil
from google.cloud import translate
from IPython.display import Image

In [2]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [4]:
input_folder = '../data/railways/germany-railways-shape/'
input_file = input_folder+'railways.shp'

output_folder = '../data/output_images/Germany/'
output_csv = output_folder+'GRM.csv'

update_folder = output_folder+'/set_3/'

#### Import routes

In [5]:
routes = gpd.read_file(input_file)

routes = routes.to_crs({'init' :'epsg:4326'})

routes.head()

,osm_id,name,type,geometry
0,2402605,None,tram,"LINESTRING (10.9021988 48.3521243, 10.9021953 ..."
1,2402606,3,tram,"LINESTRING (10.8937679 48.31041820000001, 10.8..."
2,2467882,"Stadtbahn U1, U13",rail,"LINESTRING (9.2172812 48.803314, 9.21760910000..."
3,2481116,None,rail,"LINESTRING (10.8690368 48.3937811, 10.8681289 ..."
4,2571242,Staudenbahn,preserved,"LINESTRING (10.5939151 48.146509, 10.5935423 4..."


In [6]:
routes.shape

(60612, 4)

In [5]:
routes['type'].value_counts()

rail                46835
tram                 4360
light_rail           2396
abandoned            2196
platform             1546
disused              1495
subway                746
preserved             464
narrow_gauge          254
construction          133
turntable              55
monorail               41
station                23
halt                   13
preserved;narrow       12
rack                    9
viaduct                 4
funicular               4
footway                 3
crossing                3
level_crossing          3
spur                    3
<unterschiedlich        3
service                 3
plattform               2
incline                 2
peoplemover             1
abandoned;light_        1
Brexbachtalbahn         1
container_termin        1
Name: type, dtype: int64

In [7]:
routes = routes[routes['type']=='light_rail']
routes.head()

,osm_id,name,type,geometry
5,2588600,"S3, S31",light_rail,"LINESTRING (9.872649300000001 53.473188, 9.862..."
171,4158975,None,light_rail,"LINESTRING (8.403577 48.9847996, 8.4036445 48...."
288,4242974,None,light_rail,"LINESTRING (8.4029946 49.08933, 8.402808200000..."
355,4251699,None,light_rail,"LINESTRING (8.401653100000003 48.989128, 8.402..."
356,4251700,None,light_rail,"LINESTRING (8.398828200000001 48.9905039, 8.40..."


#### Get points from routes

In [11]:
'''
Get all points
'''

points = []
rows = routes.index.tolist()

for i in rows:
    try:
        route = routes.loc[i]
#         print(route.geometry.geom_type)
        g = route.geometry
        sec_points = mapping(g)["coordinates"]
        for j in sec_points:
            tmp = (j[0],j[1])
            points.append(tmp)
    except Exception as e:
        print("Skipped route number",i,'because',e)        
        
    
print(len(points),'points')   

14032 points


In [12]:
'''
Open csv with current points
'''

curr_df = pd.read_csv(output_csv)
curr_df.head()

,Name,Longitude,Latitude,Catenary
0,10.0131888_53.5214443,10.013189,53.521444,1
1,10.0465429_53.5869819,10.046543,53.586982,1
2,10.1325456_53.4974332,10.132546,53.497433,0
3,12.3158729_51.320704,12.315873,51.320704,1
4,12.3881185_51.0477387,12.388118,51.047739,0


In [16]:
'''
Get subselection of points excuding current points
'''
curr_points = list(zip(curr_df['Longitude'].tolist(),curr_df['Latitude'].tolist()))
# curr_points

route_points = []
for point in range(0,len(points),135):
    tmp = points[point]
    if tmp not in curr_points:
        route_points.append(tmp)

print(len(route_points))

102


#### Create working dataset

In [17]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,8.4085251_49.1204456,8.408525,49.120446
1,8.5798809_48.8132501,8.579881,48.813250
2,8.4499151_48.8947733,8.449915,48.894773
3,13.3106619_52.480792,13.310662,52.480792
4,13.3861565_52.5205316,13.386157,52.520532


In [19]:
'''
Export points
'''

# Update
with open(output_csv, 'a') as f:
    df.to_csv(f, header=False,index=None)

# Write new
# df.to_csv(output_csv,header=True,index=None)
# df.to_csv('tmp.csv',header=True,index=None)

In [18]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))

# Set map centerpoint
coords = marker_points[int((len(marker_points)/2))]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 7
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [30]:
'''
Get satellite preview for image
'''

row = 1

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [ ]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 